In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
os.chdir('/home/adam/adam/causal_inference')

from causal_inference.make_data.make_raw_data import UseCaseLoader

from causal_inference.make_data.make_proning_sessions import *

In [ ]:
dl = UseCaseLoader()

In [ ]:
path = '/home/adam/adam/data/causal_inference/data/raw/position_measurements.csv'
dl.get_position_measurements(path=path)

In [ ]:
path = '/home/adam/adam/data/causal_inference/data/raw/position_measurements.csv'
df = make_proning_sessions(path=path, n_of_batches=5)

In [ ]:
dl.get_observations()